In [11]:
import os, random

In [12]:
from Crypto.Cipher import AES

In [13]:
iv = os.urandom(64)[:16]
key = os.urandom(64)[:16]

In [14]:
def inc_iv(iv):
    return int.to_bytes((int.from_bytes(iv, byteorder='big') + 1), length=16, byteorder='big')

In [15]:
def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

In [16]:
class Oracle:
    def __init__(self):
        self.iv = os.urandom(64)[:16]
        self.key = os.urandom(64)[:16]
    
    def get(self, message):
        # Increment IV on each subsequent call to oracle
        self.iv = inc_iv(self.iv)
        
        aes = AES.new(key, AES.MODE_CBC, self.iv)
        return aes.encrypt(message)
    
    def get_iv(self):
        return self.iv
    
    def challenge(self, m1, m2):
        msg = random.choice([m1, m2])
        self.iv = inc_iv(self.iv)

        aes = AES.new(key, AES.MODE_CBC, self.iv)
        return aes.encrypt(msg)

1. Send a first message to the Oracle

In [17]:
oracle = Oracle()

In [18]:
m = bytes([0 for i in range(16)])

In [19]:
c = oracle.get(m)

2. Predict the next IV, knowing that iv_{i+1} = iv_{i} + 1

In [20]:
iv0 = oracle.get_iv()

In [21]:
iv1 = inc_iv(iv0)

3. Prepare the malicious messages

In [22]:
m0 = bytes([1 for i in range(16)])

In [23]:
m1 = byte_xor(iv0, iv1)

4. Perform the challenge

In [24]:
c1 = oracle.challenge(m0, m1)

In [25]:
c1

b'.\xf1\xf6\x0b\xd9|v\x83J\x91\xc0\x8dqD\xd5\x8b'

In [26]:
def distinguisher(c, c1):
    if c == c1: return "m1"
    return "m0"

In [27]:
distinguisher(c, c1)

'm0'